In [1]:
import os
import sys
current_wd = os.getcwd()
linux_compat = current_wd.replace("\\", "/")
project_folder = linux_compat.replace("/job/work", "")
os.chdir(project_folder)

In [2]:
from magic_job import validate_datafolder, get_data
from creation import create_dataset
from prediction import Xy_split
from set_up.league_data import seasons
import pandas as pd
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import (
    GradientBoostingClassifier,
    RandomForestClassifier,
    AdaBoostClassifier,
)
from sklearn.model_selection import GridSearchCV

INFO:root:Import completed


In [3]:
estimators = {
    # "lda":{
    #     "model": LinearDiscriminantAnalysis(),
    #     "parameters": {"solver": ["svd"]},
    # },
    # "knn":{
    #     "model": KNeighborsClassifier(),
    #     "parameters": {"n_neighbors": np.arange(1, 298, 11)},
    # },
    "gb":{
        "model": GradientBoostingClassifier(),
        "parameters": {
            "n_estimators": [50, 100, 150],
            "learning_rate": [0.1, 0.3],
            "max_depth": [3, 5],
            "subsample": [0.75, 1.0],
            "max_features": [1.0, 0.75],
            "random_state": [66],
        },
    },
    "rf": {
        "model": RandomForestClassifier(),
        "parameters": {
            "n_estimators": [100],
            "max_depth": [None, 3, 5],
            "max_features": ["sqrt", 1.0],
            "random_state": [66],
        },
    },
    "ada": {
        "model": AdaBoostClassifier(),
        "parameters": {
            "n_estimators": [50, 100, 150],
            "algorithm": ["SAMME"],
            "random_state": [66],
        },
    }
}

In [ ]:
def feature_combination(df):
    feature_set = []
    cols = ['giornata', 
            'anno', 
            'posizione_casa',
            'posizione_trasferta',
            'gol_fatti_casa', 
            'gol_fatti_trasferta', 
             'squadra']
    feature_set.append(cols)
    for t in range(1, 6):
        cols += [
            f"gol_fatti_{t}_casa", 
            f"gol_fatti_{t}_trasferta",
            f"gol_subiti_{t}_casa",
            f"gol_subiti_{t}_trasferta",
            f"esito_{t}_casa",
            f"esito_{t}_trasferta",
        ]
        feature_set.append(cols)
    return feature_set

In [5]:
df.columns

Index(['giornata', 'anno', 'gol_fatti_casa', 'posizione_casa',
       'gol_fatti_1_casa', 'gol_subiti_1_casa', 'esito_1_casa',
       'gol_fatti_2_casa', 'gol_subiti_2_casa', 'esito_2_casa',
       'gol_fatti_3_casa', 'gol_subiti_3_casa', 'esito_3_casa',
       'gol_fatti_4_casa', 'gol_subiti_4_casa', 'esito_4_casa',
       'gol_fatti_5_casa', 'gol_subiti_5_casa', 'esito_5_casa',
       'gol_fatti_trasferta', 'posizione_trasferta', 'gol_fatti_1_trasferta',
       'gol_subiti_1_trasferta', 'esito_1_trasferta', 'gol_fatti_2_trasferta',
       'gol_subiti_2_trasferta', 'esito_2_trasferta', 'gol_fatti_3_trasferta',
       'gol_subiti_3_trasferta', 'esito_3_trasferta', 'gol_fatti_4_trasferta',
       'gol_subiti_4_trasferta', 'esito_4_trasferta', 'gol_fatti_5_trasferta',
       'gol_subiti_5_trasferta', 'esito_5_trasferta', 'squadra'],
      dtype='object')

In [4]:
def tune_model(X, y, target):
    scores = features_weights = pd.DataFrame()
    for algorithm, metadata in estimators.items():
        gscv = GridSearchCV(
            estimator=metadata["model"],
            param_grid=metadata["parameters"],
            scoring="accuracy",
            cv=4,
            verbose=1,
            n_jobs=2,
        )
        gscv.fit(X, y)
        gscv.best_estimator_.feature_importances_
        report = pd.DataFrame(gscv.cv_results_)
        report = report[[
            "params", 
            "mean_test_score",
            "std_test_score",
        ]]
        report["estimator"] = algorithm
        scores = pd.concat([scores, report], ignore_index=True)

        best_est = gscv.best_estimator_
        feature_importances = best_est.feature_importances_
        feature_names = best_est.feature_names_in_
        features = pd.DataFrame(feature_importances, index=feature_names, columns=[algorithm])
        features_weights = pd.concat([features_weights, features], axis=1)
    return scores, features_weights

In [3]:
validate_datafolder(seasons)
df = get_data(seasons)
df = create_dataset(df)
df.groupby('anno').giornata.count()

INFO:root:Scraping from SkySport: season 2023


anno
2019    330
2020    330
2021    330
2022    330
Name: giornata, dtype: int64

### Gf

In [7]:
X, y = Xy_split(df, "GG-NG")
scores, features = tune_model(X, y, "Gf")


GG-NG
Fitting 4 folds for each of 48 candidates, totalling 192 fits
Fitting 4 folds for each of 6 candidates, totalling 24 fits
Fitting 4 folds for each of 3 candidates, totalling 12 fits


In [8]:
features.mean(axis=1).sort_values()

gol_subiti_2_trasferta    0.020578
gol_subiti_5_trasferta    0.020732
gol_fatti_5_casa          0.021560
gol_subiti_4_trasferta    0.022679
gol_fatti_1_casa          0.023196
gol_fatti_4_casa          0.024880
gol_subiti_3_trasferta    0.025571
gol_subiti_3_casa         0.026199
gol_fatti_2_trasferta     0.028468
gol_fatti_4_trasferta     0.028977
gol_subiti_4_casa         0.030022
gol_subiti_1_trasferta    0.031332
gol_fatti_2_casa          0.031535
gol_subiti_2_casa         0.033821
gol_subiti_5_casa         0.036875
gol_subiti_1_casa         0.037388
gol_fatti_3_trasferta     0.040636
gol_fatti_5_trasferta     0.041322
gol_fatti_1_trasferta     0.043716
posizione_trasferta       0.054183
gol_fatti_3_casa          0.072949
anno                      0.082766
posizione_casa            0.083369
giornata                  0.137246
dtype: float64

In [12]:
scores.sort_values(by="rank", ascending=False)

,params,mean_test_score,std_test_score,estimator,rank
50,"{'max_depth': 3, 'max_features': 'sqrt', 'n_es...",0.553788,0.032804,rf,57.0
51,"{'max_depth': 3, 'max_features': 1.0, 'n_estim...",0.546970,0.036772,rf,56.0
52,"{'max_depth': 5, 'max_features': 'sqrt', 'n_es...",0.540152,0.033083,rf,55.0
56,"{'algorithm': 'SAMME', 'n_estimators': 150, 'r...",0.528788,0.044768,ada,54.0
55,"{'algorithm': 'SAMME', 'n_estimators': 100, 'r...",0.528788,0.046429,ada,53.0
48,"{'max_depth': None, 'max_features': 'sqrt', 'n...",0.528788,0.039131,rf,52.0
54,"{'algorithm': 'SAMME', 'n_estimators': 50, 'ra...",0.528030,0.033359,ada,51.0
53,"{'max_depth': 5, 'max_features': 1.0, 'n_estim...",0.521212,0.042801,rf,50.0
12,"{'learning_rate': 0.1, 'max_depth': 5, 'max_fe...",0.509091,0.046994,gb,49.0
1,"{'learning_rate': 0.1, 'max_depth': 3, 'max_fe...",0.507576,0.048603,gb,48.0
